In [1]:
import numpy as np
import pandas as pd
import Forward_kinematics
import Delta_multipliers
import Function_container

DH parameters: Reference and Actual

In [2]:
DH_ref = pd.DataFrame(\
    np.array([[0.08946, 0, np.pi/2],
             [0, -0.425, 0],
             [0, -0.3922, 0],
             [0.1091, 0, np.pi/2],
             [0.09465, 0, -np.pi/2],
             [0.0823, 0, 0]]
             ),
    columns = ["d", "a", "alpha"],
    index = ["q1", "q2", "q3", "q4", "q5", "q6"]
)
DH_ref.head(6)

DH_act = pd.DataFrame(\
    np.array([[0.09, 0, np.pi/2 + np.pi/20],
             [0, -0.43, 0],
             [0, -0.4, 0],
             [0.1, 0, np.pi/2 - np.pi/20],
             [0.1, 0, -np.pi/2 + np.pi/20],
             [0.1, 0, 0]]
             ),
    columns = ["d", "a", "alpha"],
    index = ["q1", "q2", "q3", "q4", "q5", "q6"]
)
print("DH_ref:\n",DH_ref,"\n")
print("DH_act:\n",DH_act)

DH_ref:
           d       a     alpha
q1  0.08946  0.0000  1.570796
q2  0.00000 -0.4250  0.000000
q3  0.00000 -0.3922  0.000000
q4  0.10910  0.0000  1.570796
q5  0.09465  0.0000 -1.570796
q6  0.08230  0.0000  0.000000 

DH_act:
        d     a     alpha
q1  0.09  0.00  1.727876
q2  0.00 -0.43  0.000000
q3  0.00 -0.40  0.000000
q4  0.10  0.00  1.413717
q5  0.10  0.00 -1.413717
q6  0.10  0.00  0.000000


Forward kinematics:

In [3]:
q0 = [np.pi/3, np.pi/3, np.pi/3, np.pi/6, np.pi/6, np.pi/2]
T_ref = Forward_kinematics.h_transf(DH_ref,q0)
T_act = Forward_kinematics.h_transf(DH_act,q0)

In [14]:
print(np.array(T_act).shape)

(6, 4, 4)


In [4]:
print(T_ref)

[array([[ 5.00000000e-01, -5.30287619e-17,  8.66025404e-01,
         0.00000000e+00],
       [ 8.66025404e-01,  3.06161700e-17, -5.00000000e-01,
         0.00000000e+00],
       [ 0.00000000e+00,  1.00000000e+00,  6.12323400e-17,
         8.94600000e-02],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]), array([[ 2.50000000e-01, -4.33012702e-01,  8.66025404e-01,
        -1.06250000e-01],
       [ 4.33012702e-01, -7.50000000e-01, -5.00000000e-01,
        -1.84030398e-01],
       [ 8.66025404e-01,  5.00000000e-01,  6.12323400e-17,
        -2.78600797e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]]), array([[-2.50000000e-01, -4.33012702e-01,  8.66025404e-01,
        -8.20000000e-03],
       [-4.33012702e-01, -7.50000000e-01, -5.00000000e-01,
        -1.42028166e-02],
       [ 8.66025404e-01, -5.00000000e-01,  6.12323400e-17,
        -6.18255960e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
 

Jacobian:

In [11]:
J = Delta_multipliers.Jacobian(T_act, DH_ref)
print(J.shape)

(6, 24)


Calculation of error vector:

In [6]:
#Translation error:
dN = np.array([\
    T_ref[5][0,3] - T_act[5][0,3],
    T_ref[5][1,3] - T_act[5][1,3],
    T_ref[5][2,3] - T_act[5][2,3]
]).T
#Rotation error:
dN1 = np.array([\
    Function_container.angle(T_act[5][0:3,0], T_ref[5][0:3,0]),
    Function_container.angle(T_act[5][0:3,1], T_ref[5][0:3,1]),
    Function_container.angle(T_act[5][0:3,2], T_ref[5][0:3,2])
]).T
#Result vector:
delta = np.vstack((dN,dN1)).reshape(6,1)

Calculation of kinematic error parameters and modified DH parameters:

In [10]:
X = np.linalg.pinv(J)@delta
print(np.linalg.pinv(J))
#print(X)
q_mod = [q0[i] - X[i,0] for i in range(6)]
DH_mod = DH_act.copy()
DH_mod['d']-=X[6:12,0]
DH_mod['a']-=X[12:18,0]
DH_mod['alpha']-=X[18:24,0]
DH_mod.head(6)

[[-0.02687263 -0.04380761 -0.02917599  0.18447389 -0.11579148 -0.12363192]
 [-0.02719813  0.08860751  0.02498893  0.12895338  0.17472242  0.01139492]
 [-0.0024723   0.09537688  0.04241259  0.17707272 -0.32661548 -0.04403672]
 [ 0.05122862 -0.11589567  0.0006906  -0.11582457 -0.19517383  0.01825059]
 [ 0.03095725 -0.10121728  0.03671257  0.1755203   0.10258832  0.02984455]
 [-0.03398347  0.06623243  0.05190111 -0.18061696 -0.1478679  -0.10441743]
 [ 0.15676996 -0.06746208 -0.09503207  0.01934868  0.07507877  0.03085534]
 [ 0.14011197  0.12214965 -0.00781517 -0.01791868  0.03438373  0.05880639]
 [ 0.17257605 -0.18754724 -0.03675779  0.01832164  0.06954909  0.01997416]
 [-0.12462476 -0.13661392  0.03371348  0.01305431 -0.03670708 -0.05529347]
 [ 0.16763175  0.0611219   0.02692826 -0.02052922  0.03553474  0.0591723 ]
 [-0.15938225 -0.07032768 -0.09490662  0.03564271 -0.01307828 -0.06134306]
 [ 0.0984919   0.05111021  0.1890075  -0.05078717 -0.02999113  0.04514731]
 [ 0.09040456 -0.10154301

,d,a,alpha
q1,0.060059,-0.001681,1.666366
q2,-0.012136,-0.440587,-0.021686
q3,-0.036708,-0.400573,0.006575
q4,0.110330,0.034206,1.338178
q5,0.081992,-0.001833,-1.413777
q6,0.113188,0.016662,-0.002779


Comparison error between initial and modified models:

In [64]:
T_mod = Forward_kinematics.h_transf(DH_mod,q_mod)
print("Initial model translation error", np.linalg.norm(dN))
print("Initial model rotation error", np.linalg.norm(dN1))

#Modified model errors:
#Translation error:
dN = np.array([\
    T_ref[5][0,3]-T_mod[5][0,3],
    T_ref[5][1,3]-T_mod[5][1,3],
    T_ref[5][2,3]-T_mod[5][2,3]
]).T
#Rotation error:
dN1 = np.array([\
    Function_container.angle(T_mod[5][0:3,0], T_ref[5][0:3,0]),
    Function_container.angle(T_mod[5][0:3,1], T_ref[5][0:3,1]),
    Function_container.angle(T_mod[5][0:3,2], T_ref[5][0:3,2])
]).T
print("\nModified model translation error", np.linalg.norm(dN))
print("Modified model rotation error", np.linalg.norm(dN1))

Initial model translation error 0.08998687849425394
Initial model rotation error 0.26820241660723193

Modified model translation error 0.11752410949239533
Modified model rotation error 0.3649303150154745
